In [239]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tqdm import tqdm
import joblib

In [240]:
IMAGE_FOR_MATCHING_PATH = 'DatasetForMatching'
IMAGE_FOR_PREDICTION_PATH = 'DatasetForPredict/10000forPredict.jpg'

In [241]:
def preprocess_image(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.GaussianBlur(img, (5, 5), 0)
    img = cv2.equalizeHist(img)
    return img

In [242]:
def compute_match_percentage(img1, img2, feature_detector):
    """Compute the match percentage between two images using keypoints and RANSAC."""
    
    # Detect keypoints and compute descriptors for both images
    keypoints1, descriptors1 = feature_detector.detectAndCompute(img1, None)
    keypoints2, descriptors2 = feature_detector.detectAndCompute(img2, None)

    if isinstance(feature_detector, cv2.ORB):  # ORB uses binary descriptors (Hamming distance)
        bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
        matches = bf.match(descriptors1, descriptors2)
    else:
        # For SIFT or other detectors, use FLANN (descriptors are float32)
        index_params = dict(algorithm=1, trees=5)
        search_params = dict(checks=50)
        flann = cv2.FlannBasedMatcher(index_params, search_params)
        matches = flann.knnMatch(descriptors1, descriptors2, k=2)

    # Lowe's ratio test (only for FLANN-based matching)
    if not isinstance(feature_detector, cv2.ORB):
        good_matches = [m for m, n in matches if m.distance < 0.7 * n.distance]
    else:
        good_matches = matches  # No need for ratio test in BFMatcher with crossCheck=True

    # Estimate homography using RANSAC if there are enough good matches
    valid_matches = 0
    if len(good_matches) >= 4:
        src_pts = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        _, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
        if mask is not None:
            valid_matches = sum(mask.ravel())

    total_keypoints = len(keypoints1)
    match_percentage = (valid_matches / total_keypoints * 100) if total_keypoints > 0 else 0
    return len(keypoints1), valid_matches, match_percentage


In [243]:
model = tf.keras.models.load_model('fcnn_model.h5')
# model = joblib.load('random_forest_model.pkl')
loaded_encoder = joblib.load('label_encoder.pkl')

In [244]:
# Fungsi untuk menggabungkan fitur dan menggunakan model untuk prediksi
def generate_prediction(matching_images_path):
    matching_images = {}
    
    # Memuat gambar untuk pencocokan
    for f in os.listdir(IMAGE_FOR_MATCHING_PATH):
        if f.endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(matching_images_path, f)
            preprocessed_image = preprocess_image(img_path)
            matching_images[os.path.basename(f)] = preprocessed_image

    # Proses gambar dalam dataset

    img_path = IMAGE_FOR_PREDICTION_PATH
    img_preprocessed = preprocess_image(img_path)
    img = cv2.imread(img_path)

    # Mengumpulkan metadata gambar
    img_name = os.path.basename(img_path)
    height, width = img.shape[:2]
    total_pixels = height * width
    avg_color_R, avg_color_G, avg_color_B = img.mean(axis=(0, 1)).tolist()

    # Hasil pencocokan SIFT
    sift_match_results = {}
    for match_name, match_img in matching_images.items():
        _, _, match_percentage = compute_match_percentage(img_preprocessed, match_img, feature_detector=cv2.SIFT_create())
        sift_match_results[f"sift_match_percentage_with_{match_name}"] = match_percentage

    # Hasil pencocokan ORB
    orb_match_results = {}
    for match_name, match_img in matching_images.items():
        _, _, match_percentage = compute_match_percentage(img_preprocessed, match_img, feature_detector=cv2.ORB_create())
        orb_match_results[f"orb_match_percentage_with_{match_name}"] = match_percentage

    # Menggabungkan semua fitur
    features = {
        "total_pixels": total_pixels,
        "avg_color_R": avg_color_R,
        "avg_color_G": avg_color_G,
        "avg_color_B": avg_color_B,
        **sift_match_results,
        **orb_match_results
    }

    # Memasukkan fitur ke dalam model dan mendapatkan prediksi
    input_features = np.array(list(features.values()), dtype=np.float32).reshape(1, -1)
    prediction = model.predict(input_features)
    predicted_index = np.argmax(prediction)
    print(predicted_index)
    print(f"{img_name}: {loaded_encoder.inverse_transform([predicted_index])}")

In [245]:
print("Generating prediction for DatasetForPredict...")
generate_prediction(IMAGE_FOR_MATCHING_PATH)

Generating prediction for DatasetForPredict...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
6
10000forPredict.jpg: [100000]
